## DnD Great Weapon Master Comparison

#### Helper Functions

In [ ]:
# Function definitions

import random as rand
import pandas as pd

ROLLS_TO_MAKE = 1_000_000

def roll(die=20):
    result = rand.randrange(1,die + 1)
    return result

def to_hit(dc=10, d20s=1):
    hit = False
    crit = False
    rolls = []
    for i in range(d20s):
        res = roll(20)
        rolls.append(res)
        if res == 20:
            hit = True
            crit = True
        elif res >= dc:
            hit = True
       
    return(hit, crit, rolls)

def damage(crit=False, sides=8, mod=4, dice=1):
    res = 0
    if crit:
        dice = dice * 2
    for i in range(dice):
        res += roll(sides)
    res += mod
    return res

def attack(dc=10, advantage=False, sup=False, dmg_die=10, dmg_mod=0, dmg_dice=1):
    dice_to_roll = 1
    if advantage:
        dice_to_roll = 2
    if sup:
        dice_to_roll = 3
   
    hit, critical, rolls = to_hit(dc, dice_to_roll)
    if hit:
        dmg = damage(crit=critical, sides=dmg_die, mod=dmg_mod, dice=dmg_dice)
    else:
        dmg = 0

    roll_type = "Normal"
    if advantage and not sup:
        roll_type = "Advantage"
    elif advantage and sup:
        roll_type= "Super"

    return {"rolls": rolls, "hit": hit, "damage": dmg, "crit": critical, "advantage": advantage, "super_advantage": sup, "roll_type": roll_type}


#### Exploration

##### 2024 Rules

In [ ]:
# 2024 Normal Rolls
new_norm = []
for i in range(ROLLS_TO_MAKE):
    new_norm.append(attack(10, False, False, 8, 7, 1))

new_norm_df = pd.DataFrame(new_norm)
print(f"2024 Normal Rolls {new_norm_df.shape}")
print(f"Damage per roll: {new_norm_df['damage'].mean()}")
print(new_norm_df.head())

# 2024 Advantage Rolls
new_adv = []
for i in range(ROLLS_TO_MAKE):
    new_adv.append(attack(10, True, False, 8, 7, 1))

new_adv_df = pd.DataFrame(new_adv)
print(f"\n2024 Advantage Rolls {new_adv_df.shape}")
print(f"Damage per roll: {new_adv_df['damage'].mean()}")
print(new_adv_df.head())

# 2024 Super Advantage Rolls
new_sup = []
for i in range(ROLLS_TO_MAKE):
    new_sup.append(attack(10, True, True, 8, 7, 1))

new_sup_df = pd.DataFrame(new_sup)
print(f"\n2024 Super Advantage Rolls {new_sup_df.shape}")
print(f"Damage per roll: {new_sup_df['damage'].mean()}")
print(new_sup_df.head())



##### 2014 Rules

In [ ]:
# 2014 Normal Rolls
old_norm = []
for i in range(ROLLS_TO_MAKE):
    old_norm.append(attack(15, False, False, 8, 14, 1))

old_norm_df = pd.DataFrame(old_norm)
print(f"2014 Normal Rolls {old_norm_df.shape}")
print(f"Damage per roll: {old_norm_df['damage'].mean()}")
print(old_norm_df.head())

# 2024 Advantage Rolls
old_adv = []
for i in range(ROLLS_TO_MAKE):
    old_adv.append(attack(15, True, False, 8, 14, 1))

old_adv_df = pd.DataFrame(old_adv)
print(f"\n2014 Advantage Rolls {old_adv_df.shape}")
print(f"Damage per roll: {old_adv_df['damage'].mean()}")
print(old_adv_df.head())

# 2024 Super Advantage Rolls
old_sup = []
for i in range(ROLLS_TO_MAKE):
    old_sup.append(attack(15, True, True, 8, 14, 1))

old_sup_df = pd.DataFrame(old_sup)
print(f"\n2014 Super Advantage Rolls {old_sup_df.shape}")
print(f"Damage per roll: {old_sup_df['damage'].mean()}")
print(old_sup_df.head())

#### Impressions

In [ ]:
new_norm_hit_perc = new_norm_df[new_norm_df['hit']].size / new_norm_df.size * 100
old_norm_hit_perc = old_norm_df[old_norm_df['hit']].size / old_norm_df.size * 100
print(f"2014 vs 2024 Normal Roll Hit : {old_norm_hit_perc:.2f}% => {new_norm_hit_perc:.2f}% \
for {old_norm_df['damage'].mean():.2f} => {new_norm_df['damage'].mean():.2f} damage.")

new_adv_hit_perc = new_adv_df[new_adv_df['hit']].size / new_adv_df.size * 100
old_adv_hit_perc = old_adv_df[old_adv_df['hit']].size / old_adv_df.size * 100
print(f"2014 vs 2024 Advantage Roll Hit : {old_adv_hit_perc:.2f}% => {new_adv_hit_perc:.2f}% \
for {old_adv_df['damage'].mean():.2f} => {new_adv_df['damage'].mean():.2f} damage.")

new_sup_hit_perc = new_sup_df[new_sup_df['hit']].size / new_sup_df.size * 100
old_sup_hit_perc = old_sup_df[old_norm_df['hit']].size / old_sup_df.size * 100
print(f"2014 vs 2024 Super Advantage Roll Hit : {old_sup_hit_perc:.2f}% => {new_sup_hit_perc:.2f}% \
for {old_sup_df['damage'].mean():.2f} => {new_sup_df['damage'].mean():.2f} damage.")


#### Analysis

In [ ]:
# Get Consolidated Data Set
old_frames = [old_norm_df, old_adv_df, old_sup_df]
new_frames = [new_norm_df, new_adv_df, new_sup_df]

old_df = pd.concat(old_frames)
new_df = pd.concat(new_frames)

old_df['rules'] = '2014'
new_df['rules'] = '2024'

all_df = pd.concat([old_df, new_df])
sample_df = all_df.sample(10)
sample_df.head(10)

#### Viaualizations

In [ ]:
# Drops some columns to clean up the DataFrame
#all_df.drop(['advantage', 'super_advantage', 'rolls'], axis=1, inplace=True)
#all_df.sample(10)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

cat_1 = sns.catplot(data=all_df, x='rules', y='hit', hue='roll_type', kind='point')
cat_2 = sns.catplot(data=all_df, x='rules', y='damage', hue='roll_type', kind='box')

plt.show()

In [ ]:
# Setup a couple of new DataFrames to sum the damage
old_df = all_df[all_df['rules'] == '2014'].copy()
old_df['cumulative'] = old_df['damage'].cumsum()
old_df.head()

new_df = all_df[all_df['rules'] == '2024'].copy()
new_df['cumulative'] = new_df['damage'].cumsum()
new_df.head()


In [ ]:
old_norm_dmg = old_df[old_df['roll_type'] == 'Normal']['cumulative']
old_adv_dmg = old_df[old_df['roll_type'] == 'Advantage']['cumulative']
old_sup_dmg = old_df[old_df['roll_type'] == 'Super']['cumulative']

new_norm_dmg = new_df[old_df['roll_type'] == 'Normal']['cumulative']
new_adv_dmg = new_df[old_df['roll_type'] == 'Advantage']['cumulative']
new_sup_dmg = new_df[old_df['roll_type'] == 'Super']['cumulative']

plt.plot(old_norm_dmg, 'r')
plt.plot(old_adv_dmg, 'b')
plt.plot(old_sup_dmg, 'g')

plt.plot(new_norm_dmg, 'm')
plt.plot(new_adv_dmg, 'c')
plt.plot(new_sup_dmg, 'y')

plt.xlabel('Rolls')
plt.ylabel('Cumulative Damage')
plt.show()


In [ ]:
from cProfile import label

from matplotlib.pyplot import show


fig_1 = plt.figure(figsize=(8,30), dpi=96)
axes_1 = fig_1.add_axes([0.1, 0.1, 0.9,.09]) # Canvas needs a border so bring it in 10%
axes_1.set_xlabel('Rolls')
axes_1.set_ylabel('Damage')
axes_1.set_title('2014 vs 2024 GWM DoT')
axes_1.plot(old_norm_dmg, label='2014 Normal')
axes_1.plot(old_adv_dmg, label='2014 Advantage')
axes_1.plot(old_sup_dmg, label='2014 Super')


axes_1.legend(loc=0)
plt.show()